<a href="https://colab.research.google.com/github/PrawinSankar777/Image-Segmentation/blob/master/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!rsync -av /content/drive/My\ Drive/agv/Stage_1 /content

In [0]:
!tar -xf Agriculture-Vision.tar.gz

In [0]:
%cd /content/drive/My Drive/agv/Stage_1/Lables/planter_skip

In [0]:
!ls -1 | wc -l

In [0]:
!pip install tifffile

Data sorting

In [0]:
!mv `ls | head -680` /content/Stage_1/Lables/weed_cluster

In [0]:
!rsync -av /content/Stage_1 /content/drive/My\ Drive/agv

In [0]:
!mv *tif /content/Stage_1/NDVI_Images/weed_cluster

In [0]:
!mv /content/Stage_1 /content/drive/My\ Drive/agv

In [0]:
import os

os.chdir("/content/Agriculture-Vision/train/labels/cloud_shadow")

In [0]:
!pip install tifffile

Import libraries


In [0]:
import os
import sys
import random

import numpy as np
import cv2
import tifffile as tff
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

## Seeding 
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

train_dir = "/content/drive/My Drive/agv/S1"
train_ids = next(os.walk(train_dir))[1]
img_size = 256

Datagen

In [0]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size, image_size):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path, id_name, "image", id_name) + ".tif"
        mask_path = os.path.join(self.path, id_name, "mask/")
        all_masks = os.listdir(mask_path)
        
        ## Reading Image
        image = tff.imread(image_path)
        image = cv2.resize(image, (self.image_size, self.image_size))
        
        mask = np.zeros((self.image_size, self.image_size, 1))
        
        ## Reading Masks
        for name in all_masks:
            _mask_path = mask_path + name
            _mask_image = cv2.imread(_mask_path, -1)
            _mask_image = cv2.resize(_mask_image, (self.image_size, self.image_size)) #128x128
            _mask_image = np.expand_dims(_mask_image, axis=-1)
            mask = np.maximum(mask, _mask_image)
            
        ## Normalizaing 
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        image = np.expand_dims(image,axis=3)
        
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

IOU Metric

In [0]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

Convolutional blocks

In [0]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

U-Net Model

In [0]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((img_size, img_size, 1))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [0]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[mean_iou])
model.summary()

In [0]:
train_ids = next(os.walk(train_dir))[1]

## Validation Data Size
val_data_size = 36

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

In [0]:
train_gen = DataGen(train_ids, train_dir, image_size=img_size, batch_size=20)
valid_gen = DataGen(valid_ids, train_dir, image_size=img_size, batch_size=5)
epochs = 10
model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=10, validation_steps=3, 
                    epochs=epochs)